<a href="https://colab.research.google.com/github/JamCourage/Tone-of-Sustainability-Report/blob/main/crawler_finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 設定路徑
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/ESG_FinBERT')  # 切換目錄(去檔案找->先掛接雲端->再複製資料夾路徑)
os.listdir()  # 看目錄內有什麼


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['finbert_example.ipynb',
 'ESG_report_110',
 'file_crawer.ipynb',
 'ESG_sent_and_class_1102_107.xlsx',
 'pdf_crawler_for_one.ipynb',
 '論文備份',
 'pdf_crawler.ipynb']

**1：整理公司股票代碼**

In [ ]:
# 1-1：擷取資料夾內所有PDF檔案名稱
pdf_file_names = os.listdir("ESG_report_110")  # 儲存資料夾內所有檔案名稱(*.pdf)為list
print(pdf_file_names[0])
print(pdf_file_names)
print(len(pdf_file_names))  # 360份

# 將 list 按照檔名中的數字部分排序
pdf_file_names = sorted(pdf_file_names, key=lambda x: int(x.split('_')[0]))
print(pdf_file_names)
print(len(pdf_file_names))

1101_110.pdf
['1101_110.pdf', '1103_110.pdf', '1104_110.pdf', '1108_110.pdf', '1216_110.pdf', '1217_110.pdf', '1218_110.pdf', '1219_110.pdf', '1227_110.pdf', '1229_110.pdf', '1236_110.pdf', '1303_110.pdf', '1301_110.pdf', '1304_110.pdf', '1305_110.pdf', '1314_110.pdf', '1319_110.pdf', '1326_110.pdf', '1402_110.pdf', '1434_110.pdf', '1455_110.pdf', '1460_110.pdf', '1473_110.pdf', '1476_110.pdf', '1477_110.pdf', '1504_110.pdf', '1513_110.pdf', '1532_110.pdf', '1598_110.pdf', '1605_110.pdf', '1702_110.pdf', '1707_110.pdf', '1709_110.pdf', '1710_110.pdf', '1711_110.pdf', '1717_110.pdf', '1722_110.pdf', '1735_110.pdf', '1762_110.pdf', '1789_110.pdf', '1904_110.pdf', '1905_110.pdf', '1907_110.pdf', '1909_110.pdf', '2002_110.pdf', '2007_110.pdf', '2014_110.pdf', '2027_110.pdf', '2029_110.pdf', '2031_110.pdf', '2059_110.pdf', '2101_110.pdf', '2103_110.pdf', '2104_110.pdf', '2105_110.pdf', '2204_110.pdf', '2207_110.pdf', '2211_110.pdf', '2303_110.pdf', '2201_110.pdf', '2312_110.pdf', '2317_110.

In [ ]:
# 1-2：要裝擷取出的文字的dataframe
import pandas as pd

# 創建一個空的 DataFrame
ESG_reports_df = pd.DataFrame(columns=['company_id', 'E_neutral', 'E_positive', 'E_negative', 'S_neutral', 'S_positive', 'S_negative', 'G_neutral', 'G_positive', 'G_negative', 'non_neutral', 'non_positive', 'non_negative'])

# 打印創建的 DataFrame
print(ESG_reports_df)


Empty DataFrame
Columns: [company_id, E_neutral, E_positive, E_negative, S_neutral, S_positive, S_negative, G_neutral, G_positive, G_negative, non_neutral, non_positive, non_negative]
Index: []


**2: 安裝FinBERT情緒分類模型 & FinBERT-ESG主題分類模型**

In [ ]:
# 2-1:
# 安裝transformers
%pip install transformers

# import套件--
# part1情緒分類
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Download and load the fine-tuned FinBERT sentiment classification model and the vocabulary (FinVocab, the finance vocabulary discussed earlier) from Hugging Face.
finbert_sent = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)  # fine-tuned FinBERT sentiment classification model
tokenizer_sent = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')  # the vocabulary (FinVocab, the finance vocabulary)

#################################################################
# part2：利用finbert_esg9classification將ESG句子分類為9大面向
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert_esg = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)  # 選用finbert之9大ESG分類模型
tokenizer_esg = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')  # pre-train
nlp_esg = pipeline("text-classification", model=finbert_esg, tokenizer=tokenizer_esg)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
# 安裝nltk tokenizer：負責斷句
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**3: 擷取PDF文字、文字前處理、斷句、FinBERT兩大模型、紀錄結果**

In [ ]:
# 3-1：讀取每一份PDF檔案
!pip install pdfplumber
import pdfplumber  # 讀取pdf套件

# 打開PDF檔案
for i, d in enumerate(pdf_file_names[350:]):  # 一個一個pdf打開 d="1102_107.pdf"
  text = ''  # 裝該pdf的純文字內容

  # 先從檔案名稱中擷取company_id
  part1, part2 = d.split('_')
  company_id = part1

  # 開始提取PDF內純文字
  path = "ESG_report_110/" + d  # 各pdf路徑
  with pdfplumber.open(path) as pdf:
    for page in pdf.pages:  # 一頁一頁看
      text += page.extract_text()  # 提取純文字內容

  # print(company_id, text[0:2000], len(text))  # check各pdf擷取狀況
  # print(company_id, len(text))

  # 將該份PDF的公司id存入dataframe中
  data_to_add = {'company_id': company_id}
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
  ESG_reports_df.loc[i, 'E_neutral'] = 0
  ESG_reports_df.loc[i, 'E_positive'] = 0
  ESG_reports_df.loc[i, 'E_negative'] = 0
  ESG_reports_df.loc[i, 'S_neutral'] = 0
  ESG_reports_df.loc[i, 'S_positive'] = 0
  ESG_reports_df.loc[i, 'S_negative'] = 0
  ESG_reports_df.loc[i, 'G_neutral'] = 0
  ESG_reports_df.loc[i, 'G_positive'] = 0
  ESG_reports_df.loc[i, 'G_negative'] = 0
  ESG_reports_df.loc[i, 'non_neutral'] = 0
  ESG_reports_df.loc[i, 'non_positive'] = 0
  ESG_reports_df.loc[i, 'non_negative'] = 0
  ################################## 3-2：刪除特殊字元，並以空白取代(保留英文字母大小寫 .,!?)############################################
  import re

  # 自定義刪除特殊字元函數，輸入值為「未清理之文字」，輸出為「已清理之文字」
  def remove_special_characters(text):
    # 定義正規表達式(regular expression)，匹配「字母以外的任何字符(特殊字元)，但保留,.!?」
    pattern = re.compile('[^a-zA-Z,.\!?]+')  # 匹配非(英文字母大小寫、正常標點符號)

    # 將正規表達式應用於文本
    cleaned_text = pattern.sub(' ', text)  # 匹配的特殊字元以空格取代

    return cleaned_text


  # 使用 apply 函數將 remove_special_characters 函數應用於 'text' 中的每個元素
  cleaned_text = remove_special_characters(text)

  if len(cleaned_text) == 0:
    continue

  ###########################################################################################################################################
  # 3-3: 斷句、分類、紀錄
  from nltk import word_tokenize, sent_tokenize

  for sent in sent_tokenize(cleaned_text):  # 斷句
    if len(sent) > 400:
      sent_r = sent[:400]  # 擷取前400字
    else:
      sent_r = sent

    sents = []
    sents.append(sent_r)  # 將句子以list形式傳入--只有一句

    ############################################### 3-4: part 1情緒分類--一句一句標情緒#######################
    # 最大不能超過512張量，否則切斷
    inputs = tokenizer_sent(sents, return_tensors="pt", padding=True, truncation=True, max_length=512)  # tokenize the sentences (pt:PyTorch輸出轉換形式):將文本 sent 轉換為 BERT 模型的輸入格式，並返回 PyTorch 張量。
    outputs = finbert_sent(**inputs)[0]  # 輸出 process them in FinBERT

    # Assign each sentence to its most probable sentiment category.
    labels = {0:'neutral', 1:'positive', 2:'negative'}

    for idx, s in enumerate(sents):  # idx=index s=sentence
      sentiment = labels[np.argmax(outputs.detach().numpy()[idx])]  # part1: neutral/positive/negative

      ################################ 3-4: part2 ESG主題分類########################################
      results_esg = nlp_esg(sents)
      esg_class_detail = results_esg[0]['label']  # part2: 九大分類

      if esg_class_detail == 'Climate Change' or  esg_class_detail == 'Natural Capital' or esg_class_detail == 'Pollution & Waste':
        esg_class = 'E'
      elif esg_class_detail == 'Human Capital' or  esg_class_detail == 'Product Liability' or esg_class_detail == 'Community Relations':
        esg_class = 'S'
      elif esg_class_detail == 'Corporate Governance' or  esg_class_detail == 'Business Ethics & Values':
        esg_class = 'G'
      else:
        esg_class = 'non_ESG'

      ############################# 3-5: 整合part1 and part2##################################
      # 共有12種情況
      # 'E_neutral', 'E_positive', 'E_negative', 'S_neutral', 'S_positive', 'S_negative', 'G_neutral', 'G_positive', 'G_negative', 'non_neutral', 'non_positive', 'non_negative'

      if esg_class == 'E' and sentiment == 'neutral':
        ESG_reports_df.loc[i, 'E_neutral'] += 1
      elif esg_class == 'E' and sentiment == 'positive':
        ESG_reports_df.loc[i, 'E_positive'] += 1
      elif esg_class == 'E' and sentiment == 'negative':
        ESG_reports_df.loc[i, 'E_negative'] += 1
      elif esg_class == 'S' and sentiment == 'neutral':
        ESG_reports_df.loc[i, 'S_neutral'] += 1
      elif esg_class == 'S' and sentiment == 'positive':
        ESG_reports_df.loc[i, 'S_positive'] += 1
      elif esg_class == 'S' and sentiment == 'negative':
        ESG_reports_df.loc[i, 'S_negative'] += 1
      elif esg_class == 'G' and sentiment == 'neutral':
        ESG_reports_df.loc[i, 'G_neutral'] += 1
      elif esg_class == 'G' and sentiment == 'positive':
        ESG_reports_df.loc[i, 'G_positive'] += 1
      elif esg_class == 'G' and sentiment == 'negative':
        ESG_reports_df.loc[i, 'G_negative'] += 1
      elif esg_class == 'non_ESG' and sentiment == 'neutral':
        ESG_reports_df.loc[i, 'non_neutral'] += 1
      elif esg_class == 'non_ESG' and sentiment == 'positive':
        ESG_reports_df.loc[i, 'non_positive'] += 1
      elif esg_class == 'non_ESG' and sentiment == 'negative':
        ESG_reports_df.loc[i, 'non_negative'] += 1
      # print(s, '------', labels[np.argmax(outputs.detach().numpy()[idx])])

# 輸出
ESG_reports_df.to_excel('ESG_sent_and_class_110.xlsx', index=True)

<ipython-input-8-6d97a83cfa50>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-6d97a83cfa50>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-6d97a83cfa50>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-6d97a83cfa50>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ESG_reports_df = ESG_reports_df.append(data_to_add, ignore_index=True)
<ipython-input-8-6d97a83cfa50>:24: F

In [ ]:
print(ESG_reports_df.loc[0, 'company_id'])
print(ESG_reports_df)

6732
  company_id E_neutral E_positive E_negative S_neutral S_positive S_negative  \
0       6732       124         21          2       248         67          9   
1       6770       589        176         27      1008        311         56   
2       6790       189         60         10       337        106         14   
3       6803       450        123         24       467        127         23   
4       6806       280         47         17       365         99         30   
5       6830        80         21          8       275         80         10   

  G_neutral G_positive G_negative non_neutral non_positive non_negative  
0       130         14          1         256           18            2  
1       471        104         19         994          112           24  
2       205         30          3         218           14            8  
3       190         30          9         287            9           12  
4       177         28          4         240           13      

In [ ]:
# 輸出
import pandas as pd
ESG_reports_df.to_excel('ESG_sent_and_class_110.xlsx', index=True)